In [11]:
import streamlit as st
import pandas as pd
from datetime import datetime,timedelta
import bs4
import requests
from pykrx import stock
import matplotlib.pyplot as plt
import numpy as np
from pymongo import MongoClient
import re
from PIL import Image
from datetime import date, timedelta

In [12]:
codes = ['035720','035420']
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']
mydb['company_price'].delete_many({})

for code in codes:
    tp_price = stock.get_market_ohlcv_by_date(
                    fromdate=datetime.strftime(
                        (datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),
                    todate=datetime.strftime(
                        (datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),
                    ticker=code,
                    adjusted=False)
    
    
    tp = stock.get_market_ohlcv_by_date(
                    fromdate=datetime.strftime(
                        (datetime.now()).date(), '%Y-%m-%d'),
                    todate=datetime.strftime(
                        (datetime.now()).date(), '%Y-%m-%d'),
                    ticker=code,
                    adjusted=False)
    tp['전일비'] = tp.loc[datetime.strftime((datetime.now()).date(), '%Y-%m-%d'),'종가']-tp_price.loc[datetime.strftime((datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),'종가']

    tp['code'] = code
    tp.reset_index(inplace=True)
    
    mydb['company_price'].insert_many(tp.to_dict('records'))

In [13]:
codes = ['1001','2001']
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']
mydb['side_price'].delete_many({})

for code in codes:
    tp_price = stock.get_index_fundamental(
                    datetime.strftime(
                        (datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),
                    datetime.strftime(
                        (datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),
                    ticker=code)
    
    
    tp = stock.get_index_fundamental(
                    datetime.strftime(
                        (datetime.now()).date(), '%Y-%m-%d'),
                    datetime.strftime(
                        (datetime.now()).date(), '%Y-%m-%d'),
                    ticker=code)
    tp['전일비'] = tp.loc[datetime.strftime((datetime.now()).date(), '%Y-%m-%d'),'종가']-tp_price.loc[datetime.strftime((datetime.now()).date()-timedelta(days=1), '%Y-%m-%d'),'종가']

    tp['code'] = code
    tp.reset_index(inplace=True)
    
    mydb['side_price'].insert_many(tp.to_dict('records'))

In [14]:
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']

companys = ['kakao','naver']
day = 3
for company in companys:
    mydb[f'{company}_article'].delete_many({})

    date_list = []
    title_list = []
    info_list = []
    url_list = []

    if company == 'kakao':
        code = '035720' # kakao
    else:
        code = '035420' # naver
    page = 1

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.32'}

    url = f'https://finance.naver.com/item/news_news.naver?code={code}&page={page}&sm=title_entity_id.basic&clusterId='
    res = requests.get(url, headers=headers)
    bs = bs4.BeautifulSoup(res.text, 'html.parser')

    while datetime.strftime(datetime.now().date() + timedelta(days=-day), '%Y-%m-%d') not in date_list:
        url = f'https://finance.naver.com/item/news_news.naver?code={code}&page={page}&sm=title_entity_id.basic&clusterId='
        res = requests.get(url, headers=headers)
        bs = bs4.BeautifulSoup(res.text, 'html.parser')

        date_list.extend([i.text.split()[0].replace('.', '-') for i in bs.find_all('td', {'class': 'date'})])
        title_list.extend([i.text.replace('\n', '').replace('[','').replace(']','') for i in bs.find_all('td', {'class': 'title'})])
        info_list.extend([i.text for i in bs.find_all('td', {'class': 'info'})])
        url_list.extend(
            ['https://finance.naver.com' + i.find('a')['href'] for i in bs.find_all('td', {'class': 'title'})])

        page += 1

    data = pd.DataFrame([date_list, title_list, info_list, url_list], index=['날짜', '제목', '정보제공', '링크']).transpose()
    data = data.drop_duplicates('제목')

    mydb[f'{company}_article'].insert_many(data.to_dict('records'))
    

In [15]:
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']
pd.DataFrame(mydb['naver_article'].find({}))

,_id,날짜,제목,정보제공,링크
0,62a2d1b8bf66dc5463da575a,2022-06-10,"네이버클라우드, ‘보안 관제 서비스’ 무료 프로모션",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
1,62a2d1b8bf66dc5463da575b,2022-06-10,"네이버, 건축가 인물정보 제공한다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
2,62a2d1b8bf66dc5463da575c,2022-06-10,"네이버, 쥬니버스쿨에 ‘사회성’ 과목 신설",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
3,62a2d1b8bf66dc5463da575d,2022-06-10,"풀무원, 맞춤 식단 '디자인밀' 네이버 스마트스토어 오픈",이데일리,https://finance.naver.com/item/news_read.naver...
4,62a2d1b8bf66dc5463da575e,2022-06-09,네이버도 만들었다…대기업들이 설립한 사내 병원,한경비즈니스,https://finance.naver.com/item/news_read.naver...
5,62a2d1b8bf66dc5463da575f,2022-06-08,"네이버 '일할 맛나는' 신사옥 기술, 묶어서 판다",비즈니스워치,https://finance.naver.com/item/news_read.naver...
6,62a2d1b8bf66dc5463da5760,2022-06-08,"(영상)네이버, 미래형공간 대중화 본격화...""공간 개념 확 바꿀 것...",이데일리,https://finance.naver.com/item/news_read.naver...
7,62a2d1b8bf66dc5463da5761,2022-06-08,"네이버 ""5G·ARC 융합기술, 내년 상용화""",서울경제,https://finance.naver.com/item/news_read.naver...
8,62a2d1b8bf66dc5463da5762,2022-06-08,"“5층까지 짓다가…” 네이버 1784, 비싼 건물된 사연",이데일리,https://finance.naver.com/item/news_read.naver...
9,62a2d1b8bf66dc5463da5763,2022-06-08,"""로봇 아닌 '이것' 팔겠다"" 네이버 밝힌 새 비즈니스는",매일경제,https://finance.naver.com/item/news_read.naver...
